Кирилл Лалаянц, R33352

# Лабораторная работа No4
## Астатизмы

Импорт необходимых для работы библиотек. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os
from tbcontrol.symbolic import routh
import math
import tqdm

SAVE_PATH = 'tex-report/figs/'
os.makedirs(SAVE_PATH, exist_ok=True)

sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")
T = sympy.Symbol("T")


def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))

Task 1

In [ ]:
def task1_tf(m1, m2):
    poly = sympy.simplify((p - m1) * (p - m2))
    coeffs = sympy.Poly(poly, p).all_coeffs()
    ss = control.tf2ss(control.tf(1, np.array(coeffs, dtype=np.float64)))
    ss_reachable = control.canonical_form(ss, form="reachable")[0]
    return ss_reachable

def task1_simulate(m1, m2, k0, k1, ts, dt, gs, y0=2, closed_loop=True):
    TF = task1_tf(m1, m2)
    u = lambda e, de: k0 * e + k1 * de
    ys = [y0]
    dys = [0]
    es = [gs[0] - ys[0]]
    des = [0]
    us = []
    for indx, t in enumerate(ts):
        if indx < 1:
            continue
        
        us.append(u(es[-1], des[-1]))
        output = control.forced_response(TF, T=[t, t + dt], U=us[-1], X0=[dys[-1], ys[-1]])
        
        ys.append(output.outputs[-1])
        dys.append((ys[-1] - ys[-2]) / dt)
        if closed_loop:
            es.append(gs[indx] - ys[-1])
            des.append((es[-1] - es[-2]) / dt)
    ys = np.array(ys)
    us = np.array(us)
    gs = np.array(gs)
    dys = np.array(dys)
    des = np.array(des)
    es = np.array(es)
    return ys, (es, gs, us, dys, des)
  

In [ ]:
dt = 0.001
m1, m2 = -1, 2

ts = get_t(10, dt=dt)
k0, k1 = 1000, 4
g = lambda t: 0 + 0 * t

task1_ys, _ = task1_simulate(m1, m2, k0, k1, ts, dt, g(ts))
plt.plot(ts[:len(task1_ys)], task1_ys, label = 'y')
plt.legend()
plt.grid()

Task 2

In [ ]:
def task2_tfd(T):
    return control.tf([1, 0], [T, 1])

def task2_simulate_TF(m1, m2, T, k0, k1, ts, gs, y0=2):
    TF = task1_tf(m1, m2)
    TFd = task2_tfd(T)
    PD = k0 + k1 * TFd
    system = control.feedback(PD * TF, 1, -1)
    system_reachable = control.canonical_form(control.tf2ss(system), form="reachable")[0]
    X0 = np.linalg.pinv(system_reachable.C) * [y0]
    return control.forced_response(system_reachable, ts, U = gs, X0 = X0).outputs

In [ ]:
W = 1 / sympy.simplify((s - m1) * (s - m2))
dW = s / (T * s + 1)
open_loop_sympy = (k0 + k1 * dW) * W
closed_loop_sympy = (open_loop_sympy / (1 + open_loop_sympy)).simplify()
denum = sympy.fraction(closed_loop_sympy)[1]
routh_matrix = routh(sympy.Poly(denum, s))
sympy.solve([e > 0 for e in routh_matrix[:, 0]], T, rational=True, simplify=True)

In [ ]:
-3 / 1996 - np.sqrt(11985) / 1996, -3 / 1996 + np.sqrt(11985) / 1996

In [ ]:
T = 10**-6
task2_r = task2_simulate_TF(m1, m2, T, k0, k1, ts, g(ts))
plt.plot(ts, task2_r, label = 'y')
plt.legend()
plt.grid()

Task 3

In [ ]:
def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real

def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1/samplerate))
    f = np.zeros(samples)
    idx = np.where(np.logical_and(freqs>=min_freq, freqs<=max_freq))[0]
    f[idx] = 1
    return fftnoise(f)

noise = band_limited_noise(0, len(ts), len(ts), len(ts))

In [ ]:
task1_ys_noise, _ = task1_simulate(m1, m2, k0, k1, ts, dt, g(ts) + noise)
plt.plot(ts[:len(task1_ys)], task1_ys, label = 'y')
plt.plot(ts[:len(task1_ys_noise)], task1_ys_noise, label = 'y')
plt.legend()
plt.grid()
np.sum((task1_ys_noise - task1_ys)**2)

In [ ]:
task2_r_noise = task2_simulate_TF(m1, m2, T, k0, k1, ts, g(ts) + noise)
plt.plot(ts[:len(task2_r)], task2_r, label = 'y')
plt.plot(ts[:len(task2_r_noise)], task2_r_noise, label = 'y')
plt.legend()
plt.grid()
np.sum((task2_r_noise - task2_r)**2)

Task 4

In [ ]:
dt = 0.001

ts = get_t(10, dt=dt)

def get_limit(K, t4_b1, t4_b0, t4_a1, t4_a0):
    b1, b0, a1, a0, k, t = sympy.symbols('b1 b0 a1 a0 k t')
    u = 2 
    U = sympy.laplace_transform(u, t, s)[0]
    W_sympy = k * (b1 * s + b0) / (s**2 + a1*s + a0)
    W_closed = W_sympy / (1 + W_sympy)


    limit = (W_closed * U * s).subs({s:0, k:K, b1: t4_b1, b0: t4_b0, a1: t4_a1, a0: t4_a0})
    return sympy.N(limit)


In [ ]:
t4_b1, t4_b0 = 1, 1
t4_a1, t4_a0 = 5, 6
K = 10

W = control.tf([t4_b1, t4_b0], [1, t4_a1, t4_a0])
W_open = K * W
W_system = control.feedback(W_open, 1, -1)
print(get_limit(K, t4_b1, t4_b0, t4_a1, t4_a0))
u = lambda t: 2 + 0 * t
y = control.forced_response(W_system, ts, u(ts)).outputs
plt.plot(ts, y, label='y')
plt.plot(ts, u(ts), label='e')
plt.legend()